<a href="https://colab.research.google.com/github/belanatal/PosPUCRio/blob/main/MVPSprintIIb_IsabelaNatal-vers%C3%A3o3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PUC Rio: Pós Graduação em Ciência de Dados e Analytics**

**MVP Sprint II: Machine Learning & Analytics (40530010056_20230_01)**
**- Parte b**

Aluna: Isabela Fernanda Natal Batista Abreu Gomes

Julho/2023

## Seção I: Introdução
**Classificador de imagens multiclasse - Paisagens Naturais**

**Contexto:** O presente trabalho tem como objetivo criar um modelo de visão computacional baseado em Aprendizado de Máquina Profundo que seja capaz de classificar uma imagem segundo uma das 6 seguintes categorias de paisagens: Prédio; Floresta; Montanha; Geleira; Rua; Mar. Os dados/imagens foram baixados da plataforma Kaggle (Para mais informações "Scene Classification": https://www.kaggle.com/datasets/nitishabharathi/scene-classification).

**Estrutura:** O notebook encontra-se dividido da seguinte forma:

- Importação das bibliotecas
- Acesso e tratamento dos dados que serão a entrada do modelo de deep learning
- Configuração do modelo de deep learning usando uma rede neural convolucional simples com Keras
- Treinamento do modelo de deep learning
- Execução do modelo de deep learning treinado
- Avaliação do modelo de deep learning
- Exportação do modelo de deep learning
- Teste do modelo exportado

### Importando as bibliotecas necessárias para executar o notebook
(Serão utilizadas as bibliotecas pandas e numpy, para a manipulação dos dados; matplotlib, para geração de gráficos; os, para manipulação de pastas e diretórios e bibliotecas voltadas para Machine e Deep Learning, tais como Keras, Tensor Flow e Scikit-Learn)

In [ ]:
!pip install tensorflow==2.13
!pip install keras

In [ ]:
# Primeiro bloco: Importação das bibliotecas e módulos
# Configuração para não exibir os warnings
import warnings
warnings.filterwarnings("ignore")
#
import os
import cv2 # Visão computacional para análise em imagens
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn.metrics as skm
from tensorflow import keras
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.losses import SparseCategoricalCrossentropy
from IPython.display import Image, display
from datetime import datetime
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import itertools
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
#
#Para utilizar o upload e download de arquivos no colab:
from google.colab import drive
from google.colab import files
import gdown
from zipfile import ZipFile

### Definindo a função plot_confusion_matrix
Para impressão da matriz de confusão na etapa de avaliação do método

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    Esta função imprime e plota a matriz de confusão.
    A normalização pode ser aplicada definindo `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Matriz de confusão normalizada")
    else:
        print('Matriz de confusão sem normalização')

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('Label real')
    plt.xlabel('Label predito')

### Acessando os dados que serão entrada do modelo - via Google Drive

In [ ]:
# Baixando a pasta compactada para o notebook
folder_path = "https://drive.usercontent.google.com/download?id=11PLDr83xGsG-lxZMxoFadgEy-iR3q4Qq&export=download&authuser=0&confirm=t&uuid=35044ed5-0d7c-4289-b486-6bf5dea852d0&at=AC2mKKQLNA-0NZzF6s1wgeK530LY:1690032546075"
output = "scene-classification.zip"
gdown.download(folder_path, output)

In [ ]:
with ZipFile('scene-classification.zip', 'r') as zip_object:
  zip_object.extractall()

In [ ]:
if os.path.exists('/content/train-scene classification'):
  os.rename('../content/train-scene classification','../content/scene')

In [ ]:
if os.path.exists('/content/scene/train'):
  os.rename('../content/scene/train','../content/scene/Imagens')

In [ ]:
os.remove('../content/scene-classification.zip')

In [ ]:
com_label=pd.read_csv('../content/scene/train.csv',sep=',')
sem_label=pd.read_csv('../content/test_WyRytb0.csv',sep=',')

In [ ]:
com_label.head()

In [ ]:
# Substituindo as categorias numéricas por categorias textuais
# (Lembrando que: 0:"Edificio", 1:"Floresta", 2:"Geleira",3:"Montanha",4:"Mar",5:"Rua")
com_label['label'] = com_label['label'].replace({0:"Edificio", 1:"Floresta", 2:"Geleira",3:"Montanha",4:"Mar",5:"Rua"})

In [ ]:
# Reavaliando as primeiras linhas para conferir se os valores da coluna label foram alterados
com_label.head()

In [ ]:
# Verificando as dimensões dos DataFrames com as indicações das imagens correspondentes aos conjuntos com label e sem label
print(com_label.shape,sem_label.shape)

In [ ]:
# Verificando se os DataFrames contêm dados nulos
# Verify if the dataframe don't have NaN or null values
check_for_nan_comlabel = com_label.isnull().values.any()
check_for_nan_semlabel = sem_label.isnull().values.any()

print(check_for_nan_comlabel)
print(check_for_nan_semlabel)

In [ ]:
# Criando a lista para armazenar os dados e selecionar a pasta da imagem a ser carregada
arquivos_com_label = []
labels = []
arquivos_teste = []
arquivos_teste_prediction = []
imagePath = "../content/scene/Imagens/"

In [ ]:
# Mantendo na pasta de imagens apenas aquelas que contêm label associado
for i in sem_label.index:
    nameOfFile = sem_label['image_name'][i]
    if os.path.exists(imagePath+nameOfFile):
        os.remove(imagePath+nameOfFile)

In [ ]:
images=os.listdir(imagePath)

heights, widths,filesize = [], [], []

# Loop que percorre cada uma das imagens extraídas
for fname in images:

    img_shape = mpimg.imread(imagePath+fname).shape
    heights.append(img_shape[0])
    widths.append(img_shape[1])
    filesize.append(os.path.getsize(imagePath+fname))

# Criação do dataset
df = pd.DataFrame({'image_name': images, 'height': heights,
                         'width': widths, 'filesize': filesize})
print(len(df)) # para confirmação de que apenas as imagens com label (17034) foram mantidas na pasta

In [ ]:
df.head()

In [ ]:
# Mesclando ao dataframe df a coluna com o label (categoria) das imagens:

df=pd.merge(df,com_label,how="left")
df.head()

In [ ]:
fig = plt.gcf()
fig.set_size_inches(3*4, 3*4)

# Visualizando 12 imagens
for i in range(12):

    sample = np.random.choice(images)
    for j in range(0,len(df)):
      if df['image_name'][j]==sample:
        titulo=df['label'][j]
        img_path =imagePath+sample
        sp = plt.subplot(3, 4, i + 1)
        sp.axis('Off')
        img = mpimg.imread(img_path)
        plt.imshow(img)
        plt.title(titulo+"-"+df['image_name'][j])

plt.show()

## Seção II: Passos iniciais para tratamento dos dados

### Acessando e tratando os dados que serão a entrada do modelo de deep learning

In [ ]:
# Definindo o tamanho do batch e dimensão das imagens e a quantidade de épocas
batch_size = 32
img_height = 150
img_width = 150
num_classes = 6
epochs = 2

In [ ]:
# Associando as imagens ao dataset com label
for i in com_label.index:
    nameOfFile = com_label['image_name'][i]
    if os.path.exists(imagePath+nameOfFile):
        image = mpimg.imread(imagePath+nameOfFile)
        if (len(image.shape)!=3): # Verify if the image is correct
            print("A imagem N°",i,' : ',nameOfFile," não está adequada")
        else :
            image = cv2.resize(image,(img_width,img_height)) # conjunto de arquivos com label sem imagens "expúrias"
            arquivos_com_label.append(image)
            labels.append(com_label['label'][i])
print(len(arquivos_com_label))
print(len(labels))

In [ ]:
labels[0]

In [ ]:
# Convertendo o dataset de treino em array
arquivos_com_label = np.array(arquivos_com_label)
labels = np.array(labels)
print(arquivos_com_label.shape,labels.shape)

## Seção III: Configuração do Modelo de Deep Learning

### Configuração do Modelo usando uma Rede Neural Convolucional (CNN) Simples

Utilizando a biblioteca Keras, será especificada uma camada convolucional 2D com 32 filtros (depois 64, 128 e 256) e função de ativação do tipo ReLU. Na sequência é adicionada uma camada softmax com o mesmo tipo de função de ativação.

In [ ]:
model = keras.models.Sequential([
    keras.layers.Conv2D(64, 3, activation='relu', padding= "same", input_shape=(img_height,img_width,3)),
    keras.layers.MaxPooling2D((2)),
    keras.layers.Conv2D(128, 3, activation='relu', padding= "same"),
    keras.layers.Conv2D(128, 3, activation='relu', padding= "same"),
    keras.layers.MaxPooling2D((2)),
    keras.layers.Conv2D(256, 3, activation='relu', padding= "same"),
    keras.layers.Conv2D(256, 3, activation='relu', padding= "same"),
    keras.layers.Conv2D(256, 3, activation='relu', padding= "same"),
    keras.layers.MaxPooling2D((2)),
    keras.layers.Conv2D(512, 3, activation='relu', padding= "same"),
    keras.layers.Conv2D(512, 3, activation='relu', padding= "same"),
    keras.layers.Conv2D(512, 3, activation='relu', padding= "same"),
    keras.layers.MaxPooling2D((2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(num_classes, activation='softmax')
])

In [ ]:
# Resumindo o modelo que será utilizado
model.summary()

### Treinamento do modelo de deep learning

Vamos considerar o modelo Adam para otimização. Como métrica para avaliação do modelo, será usado o critério de acurácia. Além disso, por se tratar de um problema de classificação com mais de 2 categorias (6, neste problema), a perda considerada será do tipo entropia cruzada.

In [ ]:
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

### Separação das imagens com labels fornecidos em treino e validação
As imagens com categorias fornecidas conforme DataFrame treino serão divididas em treino de fato e validação

In [ ]:
# Separação do dataset com os labels em treino e teste (20% para teste)
X_train, X_test, y_train, y_test = train_test_split(df['image_name'], df['label'], test_size=0.2 ,random_state=42 , shuffle=True)

# Redivisão do dataset de treino em treino (90%) e validação (10%)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42 , shuffle=True)

In [ ]:
# Avaliando os comprimentos dos datasets de treino e validação
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape,X_val.shape,y_val.shape)
print(X_train.dtype,y_train.dtype,X_test.dtype,y_test.dtype, X_val.dtype,y_val.dtype)

In [ ]:
# Confirmando que os datasets possuem quantidades representativas de cada uma das seis categorias
unique, counts = np.unique(y_train, return_counts=True)
print('Distribuição quantitativa dos labels no dataset de treino:')
print(dict(zip(unique, counts)))
print('---------------------------------------------------------------------------')
unique, counts = np.unique(y_test, return_counts=True)
print('Distribuição quantitativa dos labels no dataset de teste:')
print(dict(zip(unique, counts)))
print('---------------------------------------------------------------------------')
unique, counts = np.unique(y_val, return_counts=True)
print('Distribuição quantitativa dos labels no dataset de validação:')
print(dict(zip(unique, counts)))

Após a divisão, os datasets de treino, teste e validação ficaram com distribuições significativas e bastante homogêneas de todas as 6 categorias de imagem.

In [ ]:
# Criando os dataframes correspondentes a X e y teste, treino e validação:
dfX_train=pd.DataFrame(X_train)
dfX_test=pd.DataFrame(X_test)
dfX_val=pd.DataFrame(X_val)

dfy_train=pd.DataFrame(y_train)
dfy_test=pd.DataFrame(y_test)
dfy_val=pd.DataFrame(y_val)

In [ ]:
dfX_train.head()

In [ ]:
dfy_train.head()

In [ ]:
# Resetando o índice original
dfX_train=dfX_train.reset_index()
dfX_test=dfX_test.reset_index()
dfX_val=dfX_val.reset_index()

dfy_train=dfy_train.reset_index()
dfy_test=dfy_test.reset_index()
dfy_val=dfy_val.reset_index()

df_train=pd.merge(dfX_train,dfy_train,how="left")
df_test=pd.merge(dfX_test,dfy_test,how="left")
df_val=pd.merge(dfX_val,dfy_val,how="left")

In [ ]:
df_train.head()

In [ ]:
# Deletando a coluna index com os índices originais
df_train=df_train.drop(columns=["index"])
df_test=df_test.drop(columns=["index"])
df_val=df_val.drop(columns=["index"])

In [ ]:
df_train.head()

In [ ]:
df_test.head(20)

In [ ]:
df_val.head()

In [ ]:
ntrain = X_train.shape[0]
nval = X_val.shape[0]
ntest = X_test.shape[0]

### Preparação dos dados
Será utilizada a classe "ImageDataGenerator", do TensorFlow, destinada a aumentar e pré-processar dados de imagem em tarefas de Aprendizado Profundo.

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator=train_datagen.flow_from_dataframe(
    df_train,imagePath,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    x_col='image_name',
    y_col='label')

val_generator=val_datagen.flow_from_dataframe(
    df_val,imagePath,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    x_col='image_name',
    y_col='label')

test_generator=test_datagen.flow_from_dataframe(
    df_test ,imagePath,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    x_col='image_name',
    y_col='label')

No presente trabalho, será usado o método flow_from_dataframe() para gerar imagens aumentadas a partir de um dataframe, que aponta para as imagens originais. O método recebe parâmetros como o dataframe, o diretório com as imagens, o tamanho do lote (batch size) e o modo de classe (neste caso, sparse, em função da quantidade de classes), entre outros.

Importante ressaltar que o pré-processamento do teste não evolve o aumento de dados.

In [ ]:
history = model.fit(train_generator, steps_per_epoch=ntrain//batch_size,epochs=epochs)

## Seção IV: Execução do Modelo de Deep Learning
Nesta seção, o modelo treinado será aplicado em cada uma das imagens do dataset de teste, indicando a qual das 6 classes  (Edificio; Floresta; Montanha; Geleira; Rua; Mar) pertence.

In [ ]:
parent_dir = imagePath
count_images = 0
class_names = ['Edificio',
               'Floresta',
               'Geleira',
               'Montanha',
               'Mar',
               'Rua']
y_pred = []
y_true = []

In [ ]:
for subdir, dirs, files in os.walk(parent_dir):
  for file in files:
    for i in range(len(df_test)):
      if file==df_test['image_name'][i]:
        label=df_test['label'][i]
        y_true.append(label)
        img_path=os.path.join(imagePath,file)
        #display(Image(filename=img_path, width=300))

        img=image.load_img(img_path, target_size=(img_height, img_width))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = x.astype('float32') / 255.0

        # Previsão
        prediction = model.predict(x)
        # Printando as saídas do modelo
        predicted_class = np.argmax(prediction[0])
        probability = prediction[0][predicted_class]
        y_pred.append(class_names[predicted_class])

        display(Image(filename=img_path, width=300))
        print("Arquivo:", file)
        print("Categoria real:", label)
        print("Categoria prevista:", class_names[predicted_class])
        print("Probabilidade:", probability)
        print("\n")

## Seção V: Avaliação do Modelo



In [ ]:
# Calculando as métricas para avaliação do modelo
acuracia = skm.accuracy_score(y_true, y_pred)
precisao = skm.precision_score(y_true, y_pred, average='weighted')
recall = skm.recall_score(y_true, y_pred, average='weighted')
f1score = skm.f1_score(y_true, y_pred, average='weighted')

print("Acurácia: ", acuracia)
print("Precisão: ", precisao)
print("Recall: ", recall)
print("F-Score: ", f1score)

### Matriz de Confusão

In [ ]:
cnf_matrix = confusion_matrix(y_true, y_pred, labels=class_names)
np.set_printoptions(precisao=2)

plt.figure()
plot_confusion_matrix(cnf_matrix,
                      classes=class_names,
                      normalize= False,
                      title='Matriz real x predição')

### Exportação do Modelo

In [ ]:
# Vamos salvar o modelo treinado para aplicação futura

# obtendo a data e hora atual
now = datetime.now()

# Definição do formato
format = '%Y-%m-%dT%H%M'

# Converter a data e hora em uma string com o formato especificado
formatted_datetime = now.strftime(format)

path_model = treinamento_modelos_dir

name_model = 'trained_model_' + formatted_datetime + '.h5'

# salvando o modelo
model.save("%s/%s" % (path_model, name_model))
print("Modelo salvo com o nome: ", name_model)

In [ ]:
# Definindo os caminhos de diretorios para o modelo exportado
treinamento_modelos_dir='/content/scene/trained_models'
if not os.path.exists(treinamento_modelos_dir):
  os.makedirs(treinamento_modelos_dir)